In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from imdb import Cinemagoer

In [10]:
movies = pd.read_csv(Path('archive\movies.csv'))

In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
# seperating year of release from title
for idx in movies.index:
    title = movies.loc[idx,"title"]
    year_of_release = re.findall(r"\(([0-9]+-?\s?)\)", title)
    if year_of_release == []:
        movies.loc[idx,"year_of_release"] = np.NaN
    else:
        movies.loc[idx,"year_of_release"] = year_of_release[-1]

In [13]:
movies.isna().sum()

movieId              0
title                0
genres               0
year_of_release    327
dtype: int64

In [14]:
movies.loc[movies["year_of_release"].isna()]

,movieId,title,genres,year_of_release
10023,32930,Category 6: Day of Destruction,Action|Drama,NaN
10613,40697,Babylon 5,Sci-Fi,NaN
15719,79607,"Millions Game, The (Das Millionenspiel)",Action|Drama|Sci-Fi|Thriller,NaN
17444,87442,"Bicycle, Spoon, Apple (Bicicleta, cullera, poma)",Documentary,NaN
22651,107434,Diplomatic Immunity (2009– ),Comedy,NaN
...,...,...,...,...
57504,192339,The Sign of Three,(no genres listed),NaN
57674,192829,Parwaaz Hai Junoon,Adventure|Children|Romance,NaN
57789,193149,¿Qué te juegas?,(no genres listed),NaN
57903,193443,Wall,(no genres listed),NaN


In [15]:
movies_new = pd.DataFrame(columns=["movieId","title","genres","year_of_release"])
movies_new.head()

,movieId,title,genres,year_of_release


In [16]:
for idx in movies.index:
    genre_list = movies.loc[idx,"genres"].split("|")
    for genre in genre_list:
        temp_df = pd.DataFrame(
            [[movies.loc[idx,"movieId"], movies.loc[idx,"title"],genre,movies.loc[idx,"year_of_release"]]],
            columns=["movieId","title","genres","year_of_release"]
            )
        movies_new = pd.concat([movies_new, temp_df], ignore_index= True)

In [17]:
movies_new.head()

,movieId,title,genres,year_of_release
0,1,Toy Story (1995),Adventure,1995
1,1,Toy Story (1995),Animation,1995
2,1,Toy Story (1995),Children,1995
3,1,Toy Story (1995),Comedy,1995
4,1,Toy Story (1995),Fantasy,1995


In [18]:
dict_genre = {'Adventure':0,'Animation':1,'Children':2, 'Comedy':3,'Fantasy':4,'Romance':5,'Drama':6,'Action':7,'Crime':8, 'Thriller':9,'Horror':10,
'Mystery':11,'Sci-Fi':12,'IMAX':13,'Documentary':14,'War':15,'Musical':16,'Western':17,'Film-Noir':18,'(no genres listed)':19}

In [19]:
movie_genre_join_table = pd.DataFrame(columns=["movieId","genreId"])
movie_genre_join_table

,movieId,genreId


In [20]:
for idx in movies_new.index:
    tag_id = dict_genre[movies_new.loc[idx,'genres']]
    movie_id = movies_new.loc[idx,"movieId"]
    temp = pd.DataFrame([[movie_id, tag_id]], columns=["movieId","genreId"])
    movie_genre_join_table = pd.concat([movie_genre_join_table, temp])

In [21]:
movie_genre_join_table.head()

,movieId,genreId
0,1,0
0,1,1
0,1,2
0,1,3
0,1,4


In [22]:
genre_set = set(movies_new["genres"].values)
genre_set

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [23]:
dict_genre[np.NAN] = dict_genre["(no genres listed)"]

In [24]:
genres = pd.DataFrame(columns=["genreId", "genre"])
for genre in genre_set:
    temp_df = pd.DataFrame([[dict_genre[genre], genre]], columns=["genreId", "genre"])
    genres = pd.concat([genres, temp_df],ignore_index = True)

In [25]:
genres = genres.sort_values(by ='genreId', ascending = 1)

In [26]:
genres = genres.reset_index()

In [27]:
genres.drop(columns=["index"],inplace = True)

In [28]:
genres.loc[genres["genre"] == "(no genres listed)","genre"] = np.NAN

In [29]:
genres

,genreId,genre
0,0,Adventure
1,1,Animation
2,2,Children
3,3,Comedy
4,4,Fantasy
5,5,Romance
6,6,Drama
7,7,Action
8,8,Crime
9,9,Thriller


In [30]:
movies_new.drop(columns = ["genres"],inplace = True)

In [31]:
movies_new = movies_new.drop_duplicates()
movies_new = movies_new.reset_index().drop(columns = ["index"])

In [32]:
movies_new.loc[movies_new["year_of_release"].isna()]

,movieId,title,year_of_release
10023,32930,Category 6: Day of Destruction,NaN
10613,40697,Babylon 5,NaN
15719,79607,"Millions Game, The (Das Millionenspiel)",NaN
17444,87442,"Bicycle, Spoon, Apple (Bicicleta, cullera, poma)",NaN
22651,107434,Diplomatic Immunity (2009– ),NaN
...,...,...,...
57504,192339,The Sign of Three,NaN
57674,192829,Parwaaz Hai Junoon,NaN
57789,193149,¿Qué te juegas?,NaN
57903,193443,Wall,NaN


In [33]:
movies_new.loc[22651,"year_of_release"] = 2009

In [34]:
def munge_title(title):
    i = title.rfind(' (')
    if i != -1:
        title = title[:i]
    for suff_word in ['The', 'A', 'An']:
        suffix = ', {}'.format(suff_word)
        if title.endswith(suffix):
            title = suff_word + ' ' + title[:-len(suffix)]
    return title

In [35]:
for idx in movies_new.index:
    movies_new.loc[idx,"title"] = munge_title(movies_new.loc[idx,"title"])

In [36]:
movies_new.head()

,movieId,title,year_of_release
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [37]:
movies_new.loc[movies_new["year_of_release"].isna()]

,movieId,title,year_of_release
10023,32930,Category 6: Day of Destruction,NaN
10613,40697,Babylon 5,NaN
15719,79607,The Millions Game,NaN
17444,87442,"Bicycle, Spoon, Apple",NaN
24089,112406,Brazil: In the Shadow of the Stadiums,NaN
...,...,...,...
57504,192339,The Sign of Three,NaN
57674,192829,Parwaaz Hai Junoon,NaN
57789,193149,¿Qué te juegas?,NaN
57903,193443,Wall,NaN


In [38]:
links = pd.read_csv(Path("archive/links.csv"))
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [41]:
## Get movie year from IMDB
imdb_year_1 = []
count=0
# create an instance of the Cinemagoer class
ia = Cinemagoer()

for x,m in zip(movies_new.loc[movies_new["year_of_release"].isna(),'movieId'],movies_new.loc[movies_new["year_of_release"].isna(),'title']):
    y = links[links['movieId']==x]['imdbId'].values[0]
     #################### DO NOT TOUCH THIS CODE #####################
    # get a movie
    movie = ia.get_movie(y)
    try:
        movies_new.loc[movies_new["movieId"] == x, 'year_of_release'] = movie['year']
    except:
        print("Id:",x,"Movie:",m,"ImdbId:",y)
        imdb_year_1.append([x,m,0])
    if (count%10==0):
        print(count,"------- Done ----------")
        print(m,movie['year'])
    count+=1
    
    #########

0 ------- Done ----------
Tatort: Im Schmerz geboren 2014
10 ------- Done ----------
La vendetta dei barbari 1960
20 ------- Done ----------
One Night Only 1984
30 ------- Done ----------
Vaastupurush 2002
40 ------- Done ----------
Chinese Boxes 1984
50 ------- Done ----------
Disaster Playground 2015
60 ------- Done ----------
Roger la Honte 1966
70 ------- Done ----------
Alone With People 2014
80 ------- Done ----------
Nocturnal Animals 2016
90 ------- Done ----------
Beauty and the Breast 2012
100 ------- Done ----------
Vergeef me 2001
110 ------- Done ----------
Pawn's Move 2011
120 ------- Done ----------
Citizen King 2004
130 ------- Done ----------
After Eden 2015
140 ------- Done ----------
Tibetana 1970
150 ------- Done ----------
Bad Dad Rehab 2016
160 ------- Done ----------
The Adventures of Cinderella's Daughter 2000
170 ------- Done ----------
Dolpo Tulku - Heimkehr in den Himalaya 2010
180 ------- Done ----------
Third Guest 2016
190 ------- Done ----------
Jedi Juni

In [42]:
movies_new.loc[movies_new["year_of_release"].isna()]

,movieId,title,year_of_release
50874,177265,Checkmate,NaN
56064,188661,Untitled Star Trek Sequel,NaN


In [43]:
movie_genre_join_table = movie_genre_join_table.reset_index().drop(columns=["index"])

In [44]:
movie_genre_join_table = movie_genre_join_table.drop_duplicates()

In [45]:
movie_genre_join_table.loc[movie_genre_join_table["genreId"] == 19]

,movieId,genreId
34699,83773,19
34728,83829,19
35071,84768,19
35672,86493,19
35973,87061,19
...,...,...
106048,193815,19
106074,193849,19
106080,193855,19
106094,193870,19


In [46]:
movies_new.rename(columns={"movieId":"movie_id"})
movies_new.to_csv(Path("preproc_datasets/movies.csv"),index=False)

In [47]:
movie_genre_join_table.rename(columns={"movieId":"movie_id","genreId":"genre_id"})
movie_genre_join_table.to_csv(Path("preproc_datasets/movie_genres.csv"),index = False)

In [48]:
genres.rename(columns = {"genreId":"genre_id"})
genres.to_csv(Path("preproc_datasets/genres.csv"))

In [ ]:
for movieid in movie_genre_join_table.loc[movie_genre_join_table["genreId"] == 19, "movieId"].values:
    imdb_id = links.loc[links["movieId"] == movieid,"imdbId"].values[0]
    try:
        data = ia.get_movie(imdb_id)
        new_genres = data["genres"]
        movie_genre_join_table.drop(movie_genre_join_table.loc[movie_genre_join_table["movieId"] == movieid].index, inplace = True)
        counter = 20
        for _ in new_genres:
            if dict_genre.get(_) is None:
                dict_genre[_] = counter
                counter += 1
                print(_)
            new_row = pd.DataFrame([[movieid,dict_genre.get(_)]],columns=["movieId","genreId"])
            movie_genre_join_table = pd.concat([new_row,movie_genre_join_table])
            
    except:
        print("no genre avail",movieid)

Short
Music
History
no genre avail 94431
Biography
no genre avail 114725
no genre avail 115419
no genre avail 115441
